In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import pandas as pd
import geopandas as gpd
import glob
from covid19_vulnerabilidad_mex.datos import arregla_cvegeo

## CENSO 2020

Primero vamos a actualizar las variables censales para utilizar las del censo 2020

In [ ]:
censo = pd.read_csv("datos/municipios/actualizacion_2020/conjunto_de_datos_iter_00CSV20.zip", dtype={'ENTIDAD':str, 'MUN':str, 'LOC':str})
censo

/home/plablo/miniconda3/envs/vulnerabilidd-covid/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,LONGITUD,LATITUD,ALTITUD,POBTOT,...,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,VPH_SINRTV,VPH_SINLTC,VPH_SINCINT,VPH_SINTIC,TAMLOC
0,00,Total nacional,000,Total nacional,0000,Total nacional,NaN,NaN,NaN,126014024,...,30775898,18307193,15211306,6616141,4047100,1788552,3170894,15108204,852871,*
1,00,Total nacional,000,Total nacional,9998,Localidades de una vivienda,NaN,NaN,NaN,250354,...,47005,8385,18981,1732,1113,12775,14143,51293,7154,*
2,00,Total nacional,000,Total nacional,9999,Localidades de dos viviendas,NaN,NaN,NaN,147125,...,25581,5027,11306,971,708,8247,10065,29741,5283,*
3,01,Aguascalientes,000,Total de la entidad Aguascalientes,0000,Total de la Entidad,NaN,NaN,NaN,1425607,...,359895,236003,174089,98724,70126,6021,15323,128996,1711,*
4,01,Aguascalientes,000,Total de la entidad Aguascalientes,9998,Localidades de una vivienda,NaN,NaN,NaN,3697,...,732,205,212,48,41,39,62,530,20,*
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195657,32,Zacatecas,058,Santa María de la Paz,0037,Mesa Grande,"103°24'52.925"" W","21°33'12.375"" N",1944.0,165,...,24,9,13,0,0,3,9,38,1,1
195658,32,Zacatecas,058,Santa María de la Paz,0039,San Isidro,"103°20'14.164"" W","21°29'11.130"" N",2031.0,1,...,*,*,*,*,*,*,*,*,*,1
195659,32,Zacatecas,058,Santa María de la Paz,0041,San Miguel Tepetitlán,"103°20'09.356"" W","21°30'15.168"" N",1977.0,97,...,21,12,21,1,1,1,10,19,0,1
195660,32,Zacatecas,058,Santa María de la Paz,0042,San Rafael,"103°22'20.134"" W","21°31'39.341"" N",2042.0,3,...,*,*,*,*,*,*,*,*,*,1


Seleccionamos las filas que corresponden a los totales municipales (de acuertdo a la metodología de INEGI)

In [ ]:
totales_municipales = censo[(censo.ENTIDAD != '00') & (censo.MUN != '000') & (censo.LOC == '0000')]
totales_municipales

,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,LONGITUD,LATITUD,ALTITUD,POBTOT,...,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,VPH_SINRTV,VPH_SINLTC,VPH_SINCINT,VPH_SINTIC,TAMLOC
6,01,Aguascalientes,001,Aguascalientes,0000,Total del Municipio,NaN,NaN,NaN,948990,...,251719,178619,130290,80951,56131,3299,7293,74227,731,*
574,01,Aguascalientes,002,Asientos,0000,Total del Municipio,NaN,NaN,NaN,51536,...,10682,4526,3882,596,556,377,1447,7206,177,*
749,01,Aguascalientes,003,Calvillo,0000,Total del Municipio,NaN,NaN,NaN,58250,...,13666,6553,4749,1382,1352,443,904,8061,138,*
918,01,Aguascalientes,004,Cosío,0000,Total del Municipio,NaN,NaN,NaN,17000,...,3424,1741,1664,223,191,139,414,1912,50,*
985,01,Aguascalientes,005,Jesús María,0000,Total del Municipio,NaN,NaN,NaN,129929,...,31408,19920,13483,9296,6582,480,1150,11447,148,*
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195372,32,Zacatecas,054,Villa Hidalgo,0000,Total del Municipio,NaN,NaN,NaN,19446,...,3929,1150,1090,109,106,160,744,3569,80,*
195422,32,Zacatecas,055,Villanueva,0000,Total del Municipio,NaN,NaN,NaN,31558,...,7418,4411,5181,643,569,306,746,4252,101,*
195542,32,Zacatecas,056,Zacatecas,0000,Total del Municipio,NaN,NaN,NaN,149607,...,39755,31989,28630,11944,7639,681,1185,8783,129,*
195616,32,Zacatecas,057,Trancoso,0000,Total del Municipio,NaN,NaN,NaN,20455,...,3834,1092,1854,204,185,130,677,3274,59,*


### Selección de variables y preproceso
Vamos a quedarnos sólo con las columnas que queremos, en este momento vamos a hacer una selección relativamente grande de variables, después iremos afinando para cada índice que calculemos.

Del censo vamos a obtener variables demógráficas y de las viviendas. Las características socieconómicas y de acceso a servicios las vamos a obtener de las bases de CONEVAL

* Población Total (POBTOT)
* Total de viviendas particulares habitadas (TVIVPARHAB)
* Hombres (POBMAS) -> Transformada en tasa por población
* Mujeres (POBFEM) -> Transformada en tasa por población
* Población > 60 años (P_60YMAS) -> Transformada en tasa por población
* Población masculina > 60 años (P_60YMAS_M) -> Transformada en tasa por población
* Población femenina > 60 años (P_60YMAS_F) -> Transformada en tasa por población
* Población > 65 años (POB65_MAS). Transformada en tasa por población
* Población 5 y más no habla español (P5_HLI_NHE) -> Transformada en tasa por población
* Población con discapacidad (PCON_DISC) -> Transformada en tasa por población
* Promedio de ocupantes en viviendas particulares habitadas (PROM_OCUP). 
* Promedio de ocupantes por cuarto en viviendas particulares habitadas (PRO_OCUP_C)
* Viviendas particulares habitadas que disponen de automóvil o camioneta (VPH_AUTOM) -> Transformada en tasa por vivienda

Al final vamos a renombrar las columnas y calcular bien la clave de municipio


In [ ]:
id_vars = ['ENTIDAD', 'NOM_ENT', 'MUN',	'NOM_MUN']
vars_poblacion = ['POBTOT', 'POBMAS', 'POBFEM', 'P_60YMAS', 'P_60YMAS_M', 'P_60YMAS_F', 
                   'POB65_MAS', 'P5_HLI_NHE', 'PCON_DISC']
vars_vivienda = ['TVIVPARHAB', 'PROM_OCUP', 'PRO_OCUP_C', 'VPH_AUTOM']
censales_municipio = totales_municipales[id_vars + vars_poblacion + vars_vivienda].copy()
censales_municipio.loc[:, vars_poblacion + vars_vivienda] = censales_municipio[vars_poblacion + vars_vivienda].astype(float)
censales_municipio.loc[:, set(vars_poblacion) - set(['POBTOT'])] = censales_municipio.loc[:, 
                                                                                          set(vars_poblacion) - set(['POBTOT'])].div(censales_municipio['POBTOT'], axis=0)
censales_municipio.loc[:, 'VPH_AUTOM'] = censales_municipio.loc[:, 'VPH_AUTOM'].div(censales_municipio['TVIVPARHAB'], axis=0)
rename_dict = {'ENTIDAD': 'entidad_cvegeo', 'NOM_ENT': 'State', 'MUN': 'municipio_cvegeo', 'NOM_MUN': 'Municipality',
               'POBTOT': 'Population', 'POBMAS': 'Male', 'POBFEM': 'Female', 'P_60YMAS': '60 and older',
               'P_60YMAS_M': '60 and older males', 'P_60YMAS_F': '60 and older females', 
               'POB65_MAS': '65 and older', 'P5_HLI_NHE': 'Non-spanish speakers', 'PCON_DISC': 'Disabled', 
               'PROM_OCUP': 'Dwellers per household', 'PRO_OCUP_C': 'Dwellers per room', 
               'VPH_AUTOM': 'Households with automobile', 'TVIVPARHAB': 'Inhabited households'}
censales_municipio = censales_municipio.rename(rename_dict, axis=1)
censales_municipio['municipio_cvegeo'] = censales_municipio['entidad_cvegeo'] + censales_municipio['municipio_cvegeo'] 
censales_municipio

,entidad_cvegeo,State,municipio_cvegeo,Municipality,Population,Male,Female,60 and older,60 and older males,60 and older females,65 and older,Non-spanish speakers,Disabled,Inhabited households,Dwellers per household,Dwellers per room,Households with automobile
6,01,Aguascalientes,01001,Aguascalientes,948990.0,0.486910,0.513090,0.108523,0.048750,0.059773,0.071593,0.000008,0.050080,266778.0,3.54,0.84,0.636087
574,01,Aguascalientes,01002,Asientos,51536.0,0.490162,0.509838,0.091140,0.044260,0.046880,0.064634,0.000000,0.056000,12542.0,4.10,1.07,0.529341
749,01,Aguascalientes,01003,Calvillo,58250.0,0.490352,0.509648,0.134403,0.064961,0.069442,0.096841,0.000017,0.070180,15546.0,3.74,0.97,0.580985
918,01,Aguascalientes,01004,Cosío,17000.0,0.487765,0.512235,0.094882,0.045706,0.049176,0.065765,0.000000,0.047353,3937.0,4.32,1.14,0.537211
985,01,Aguascalientes,01005,Jesús María,129929.0,0.494262,0.505738,0.075541,0.036274,0.039268,0.050320,0.000008,0.038760,33214.0,3.91,0.95,0.675558
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195372,32,Zacatecas,32054,Villa Hidalgo,19446.0,0.488738,0.511262,0.109328,0.054510,0.054818,0.077754,0.000000,0.072200,4951.0,3.93,1.08,0.490810
195422,32,Zacatecas,32055,Villanueva,31558.0,0.494011,0.505989,0.178655,0.086729,0.091926,0.135655,0.000000,0.067463,9045.0,3.47,0.84,0.614151
195542,32,Zacatecas,32056,Zacatecas,149607.0,0.481074,0.518926,0.125977,0.056261,0.069716,0.082677,0.000067,0.056515,42394.0,3.49,0.75,0.660660
195616,32,Zacatecas,32057,Trancoso,20455.0,0.490785,0.509215,0.087265,0.042728,0.044537,0.058519,0.000000,0.049866,4669.0,4.38,1.13,0.448918


## Rezago social 2020 (CONEVAL)

Leemos los datos y unimos la columna de rezago a los datos que calculamos arriba

In [ ]:
# censales_municipio.loc[:,set(lista_variables) - set(['POBTOT'])]
rezago_social = pd.read_csv("datos/municipios/actualizacion_2020/rezago_social_2020.csv", dtype='object')
rezago_social = arregla_cvegeo(rezago_social, campo_cvegeo='Clave municipio', extraer_ent=False)
rezago_social['Índice de rezago social'] = rezago_social['Índice de rezago social'].astype(float)
rezago_social

,Clave entidad,Entidad\nfederativa,Clave municipio,Municipio,Índice de rezago social
0,1,Aguascalientes,01001,Aguascalientes,-1.315320
1,1,Aguascalientes,01002,Asientos,-0.857301
2,1,Aguascalientes,01003,Calvillo,-0.918554
3,1,Aguascalientes,01004,Cosío,-1.004023
4,1,Aguascalientes,01005,Jesús María,-1.173361
...,...,...,...,...,...
2464,32,Zacatecas,32054,Villa Hidalgo,-0.692240
2465,32,Zacatecas,32055,Villanueva,-0.884952
2466,32,Zacatecas,32056,Zacatecas,-1.336926
2467,32,Zacatecas,32057,Trancoso,-0.748505


In [ ]:
indicadores_municipales = (censales_municipio
                           .merge(rezago_social[['Clave municipio', 'Índice de rezago social']], left_on='municipio_cvegeo', right_on='Clave municipio')
                           .drop(columns=['Clave municipio'])
                          )
indicadores_municipales

,entidad_cvegeo,State,municipio_cvegeo,Municipality,Population,Male,Female,60 and older,60 and older males,60 and older females,65 and older,Non-spanish speakers,Disabled,Inhabited households,Dwellers per household,Dwellers per room,Households with automobile,Índice de rezago social
0,01,Aguascalientes,01001,Aguascalientes,948990.0,0.486910,0.513090,0.108523,0.048750,0.059773,0.071593,0.000008,0.050080,266778.0,3.54,0.84,0.636087,-1.315320
1,01,Aguascalientes,01002,Asientos,51536.0,0.490162,0.509838,0.091140,0.044260,0.046880,0.064634,0.000000,0.056000,12542.0,4.10,1.07,0.529341,-0.857301
2,01,Aguascalientes,01003,Calvillo,58250.0,0.490352,0.509648,0.134403,0.064961,0.069442,0.096841,0.000017,0.070180,15546.0,3.74,0.97,0.580985,-0.918554
3,01,Aguascalientes,01004,Cosío,17000.0,0.487765,0.512235,0.094882,0.045706,0.049176,0.065765,0.000000,0.047353,3937.0,4.32,1.14,0.537211,-1.004023
4,01,Aguascalientes,01005,Jesús María,129929.0,0.494262,0.505738,0.075541,0.036274,0.039268,0.050320,0.000008,0.038760,33214.0,3.91,0.95,0.675558,-1.173361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2464,32,Zacatecas,32054,Villa Hidalgo,19446.0,0.488738,0.511262,0.109328,0.054510,0.054818,0.077754,0.000000,0.072200,4951.0,3.93,1.08,0.490810,-0.692240
2465,32,Zacatecas,32055,Villanueva,31558.0,0.494011,0.505989,0.178655,0.086729,0.091926,0.135655,0.000000,0.067463,9045.0,3.47,0.84,0.614151,-0.884952
2466,32,Zacatecas,32056,Zacatecas,149607.0,0.481074,0.518926,0.125977,0.056261,0.069716,0.082677,0.000067,0.056515,42394.0,3.49,0.75,0.660660,-1.336926
2467,32,Zacatecas,32057,Trancoso,20455.0,0.490785,0.509215,0.087265,0.042728,0.044537,0.058519,0.000000,0.049866,4669.0,4.38,1.13,0.448918,-0.748505


## Indicadores anteriores

Aquí vamos a savar de la tabla de indicadores que usamos antes los indicadores que queremos conservar y unirlos a nuestros nuevos datos

**Carencias sociales 2015 (CONEVAL)** 

Nota: Todas las variables son porcentajes

* pobreza -> personas en pobreza
* pobreza_e -> personas en pobreza extrema
* pobreza_m -> personas en pobreza moderada
* vul_car -> personas vulnerables por carencias sociales
* vul_ing -> personas vulnerables por ingresos
* npnv -> personas no-pobres y no vulnerables
* ic_rezedu -> personas con rezago educativo
* ic_asalud -> personas con carencias de acceso a servicios de salud
* ic_segsoc -> personas con carencia de seguridad social
* ic_cv -> carencia por calidad y espacios de vivienda
* ic_sbv -> carencia por servivios básicos en la vivienda
* ic_ali -> carencia alimentaria
* carencias -> personas con al menos una carencia social
* carencias3 -> personas con 3 carencias sociales
* plb -> personas con ingreso inferior a la línea de bienestar
* plbm -> personas con ingreso inferior a la línea de bienestar mínimo

**Hospitales (CLUES 2018)**

* total_camas_publicos -> Total de camas en hospitales públicos (Estatal)
* total_camas_privados -> Total de camas en hospitales privados (Estatal)
* total_hospitales_publicos -> Total de hospitales públicos

**Comorbilidades (INEGI 2018)**

* pct_hipertension -> personas con hipertensión
* pct_diabetes -> personas con diabetes
* pct_obesidad -> personas con obesidad

Para seguir con el estilo que estamos usando, también vamos a renombrar las variables.

**NOTA:** Estamos usando la base reclasificada, entonces ya todas las variables van a estar procesadas para que varien en la misma dirección

In [ ]:
originales = pd.read_csv("datos/municipios/indicadores_reclasificados.csv", dtype='object')
id_vars = ['cvegeo']
data_vars = ['pobreza', 'pobreza_e', 'pobreza_m', 'vul_car', 'vul_ing', 
             'npnv', 'ic_rezedu', 'ic_asalud', 'ic_segsoc', 'ic_cv', 
             'ic_sbv', 'ic_ali', 'carencias', 'carencias3', 'plb', 'plbm',
             'total_camas_publicos', 'total_camas_privados', 'total_hospitales_publicos',
             'pct_hipertension', 'pct_diabetes', 'pct_obesidad']
originales = originales[id_vars + data_vars]
originales[data_vars] = originales[data_vars].astype(float)
rename_dict = {'cvegeo': 'municipio_cvegeo', 'pobreza': 'Poverty', 'pobreza_e': 'Extreme poverty', 'pobreza_m': 'Moderate poverty',
               'vul_car': 'Vulnerable (social lacks)', 'vul_ing': 'Vulnerable (Income)','npnv': 'Not poor nor vulnerable',
               'ic_rezedu': 'Educational lag', 'ic_asalud': 'Health access lag', 'ic_segsoc': 'Social security lag', 
               'ic_cv': 'Household characteristics lag', 'ic_sbv': 'Basic services lag', 'ic_ali': 'Alimentary lag', 'carencias': 'At least one lag', 
               'carencias3': 'At least three lags', 'plb': 'People under wellness line', 'plbm': 'People under minnimum wellness line',
               'total_camas_publicos': 'Beds in public hospitals per (state) habitant', 'total_camas_privados': 'Beds in private hospitals per (state) habitant', 
               'total_hospitales_publicos': 'Number of public hospitals per (state) habitant', 
               'total_hospitales_privados': 'Number of private hospitals per (state) habitant',
               'pct_hipertension': 'People with hypertension', 'pct_diabetes': 'People with diabetes', 'pct_obesidad': 'People with obesity'}
originales = originales.rename(rename_dict, axis=1)
originales

,municipio_cvegeo,Poverty,Extreme poverty,Moderate poverty,Vulnerable (social lacks),Vulnerable (Income),Not poor nor vulnerable,Educational lag,Health access lag,Social security lag,...,At least one lag,At least three lags,People under wellness line,People under minnimum wellness line,Beds in public hospitals per (state) habitant,Beds in private hospitals per (state) habitant,Number of public hospitals per (state) habitant,People with hypertension,People with diabetes,People with obesity
0,01001,26.1,1.6,24.5,26.4,11.5,0.456193,11.7,15.1,36.4,...,52.6,7.3,37.6,10.0,0.959461,0.993091,0.981481,14.942242,7.495861,31.486541
1,01002,54.0,4.4,49.5,29.0,7.7,0.859517,19.8,6.0,69.0,...,83.0,14.1,61.7,24.4,0.959461,0.993091,0.981481,15.320425,7.953634,32.282284
2,01003,56.8,3.1,53.6,27.6,6.3,0.859517,23.2,9.9,73.6,...,84.3,10.5,63.1,26.8,0.959461,0.993091,0.981481,13.751906,9.172624,40.004293
3,01004,43.2,2.8,40.4,32.7,8.0,0.756798,14.7,5.5,62.7,...,75.9,9.9,51.2,16.6,0.959461,0.993091,0.981481,16.431493,7.383116,32.596450
4,01005,25.0,1.7,23.4,38.0,6.6,0.542296,17.3,15.2,45.0,...,63.1,11.3,31.6,7.9,0.959461,0.993091,0.981481,12.356755,6.745819,34.731715
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2452,32054,71.8,9.2,62.6,17.0,4.9,0.904834,20.8,7.7,78.8,...,88.8,16.7,76.8,38.5,0.991437,1.000000,0.824074,16.920279,9.895491,31.075955
2453,32055,61.0,6.9,54.1,23.3,7.2,0.871601,26.1,12.6,65.7,...,84.3,14.7,68.2,28.7,0.991437,1.000000,0.824074,21.746589,12.050064,30.526273
2454,32056,25.9,1.8,24.1,28.3,9.0,0.444109,10.4,14.5,40.2,...,54.2,6.5,34.9,8.5,0.991437,1.000000,0.824074,20.010453,11.814172,36.822130
2455,32057,63.1,7.7,55.4,20.6,7.6,0.868580,19.5,8.4,72.4,...,83.8,16.3,70.7,30.1,0.991437,1.000000,0.824074,15.941678,8.089966,32.477646


Unimos con los indicadores municipales

In [ ]:
indicadores_municipales = indicadores_municipales.merge(originales)
indicadores_municipales

,entidad_cvegeo,State,municipio_cvegeo,Municipality,Population,Male,Female,60 and older,60 and older males,60 and older females,...,At least one lag,At least three lags,People under wellness line,People under minnimum wellness line,Beds in public hospitals per (state) habitant,Beds in private hospitals per (state) habitant,Number of public hospitals per (state) habitant,People with hypertension,People with diabetes,People with obesity
0,01,Aguascalientes,01001,Aguascalientes,948990.0,0.486910,0.513090,0.108523,0.048750,0.059773,...,52.6,7.3,37.6,10.0,0.959461,0.993091,0.981481,14.942242,7.495861,31.486541
1,01,Aguascalientes,01002,Asientos,51536.0,0.490162,0.509838,0.091140,0.044260,0.046880,...,83.0,14.1,61.7,24.4,0.959461,0.993091,0.981481,15.320425,7.953634,32.282284
2,01,Aguascalientes,01003,Calvillo,58250.0,0.490352,0.509648,0.134403,0.064961,0.069442,...,84.3,10.5,63.1,26.8,0.959461,0.993091,0.981481,13.751906,9.172624,40.004293
3,01,Aguascalientes,01004,Cosío,17000.0,0.487765,0.512235,0.094882,0.045706,0.049176,...,75.9,9.9,51.2,16.6,0.959461,0.993091,0.981481,16.431493,7.383116,32.596450
4,01,Aguascalientes,01005,Jesús María,129929.0,0.494262,0.505738,0.075541,0.036274,0.039268,...,63.1,11.3,31.6,7.9,0.959461,0.993091,0.981481,12.356755,6.745819,34.731715
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2452,32,Zacatecas,32054,Villa Hidalgo,19446.0,0.488738,0.511262,0.109328,0.054510,0.054818,...,88.8,16.7,76.8,38.5,0.991437,1.000000,0.824074,16.920279,9.895491,31.075955
2453,32,Zacatecas,32055,Villanueva,31558.0,0.494011,0.505989,0.178655,0.086729,0.091926,...,84.3,14.7,68.2,28.7,0.991437,1.000000,0.824074,21.746589,12.050064,30.526273
2454,32,Zacatecas,32056,Zacatecas,149607.0,0.481074,0.518926,0.125977,0.056261,0.069716,...,54.2,6.5,34.9,8.5,0.991437,1.000000,0.824074,20.010453,11.814172,36.822130
2455,32,Zacatecas,32057,Trancoso,20455.0,0.490785,0.509215,0.087265,0.042728,0.044537,...,83.8,16.3,70.7,30.1,0.991437,1.000000,0.824074,15.941678,8.089966,32.477646


Guardamos un csv

In [ ]:
indicadores_municipales.to_csv("datos/municipios/actualizacion_2020/indicadores_finales.csv")

## Densidad de población

In [ ]:
muns_geo = gpd.read_file("datos/municipios/marco_2019.json")
muns_geo['area'] = muns_geo.to_crs(6362).area
indicadores_municipales = indicadores_municipales.merge(muns_geo[['municipio_cvegeo', 'area']], on='municipio_cvegeo')
indicadores_municipales['Population density'] = indicadores_municipales['Population'] / indicadores_municipales['area']
indicadores_municipales = indicadores_municipales.drop(columns=['area'])
indicadores_municipales

,entidad_cvegeo,State,municipio_cvegeo,Municipality,Population,Male,Female,60 and older,60 and older males,60 and older females,...,At least three lags,People under wellness line,People under minnimum wellness line,Beds in public hospitals per (state) habitant,Beds in private hospitals per (state) habitant,Number of public hospitals per (state) habitant,People with hypertension,People with diabetes,People with obesity,Population density
0,01,Aguascalientes,01001,Aguascalientes,948990.0,0.486910,0.513090,0.108523,0.048750,0.059773,...,7.3,37.6,10.0,0.959461,0.993091,0.981481,14.942242,7.495861,31.486541,0.000808
1,01,Aguascalientes,01002,Asientos,51536.0,0.490162,0.509838,0.091140,0.044260,0.046880,...,14.1,61.7,24.4,0.959461,0.993091,0.981481,15.320425,7.953634,32.282284,0.000094
2,01,Aguascalientes,01003,Calvillo,58250.0,0.490352,0.509648,0.134403,0.064961,0.069442,...,10.5,63.1,26.8,0.959461,0.993091,0.981481,13.751906,9.172624,40.004293,0.000063
3,01,Aguascalientes,01004,Cosío,17000.0,0.487765,0.512235,0.094882,0.045706,0.049176,...,9.9,51.2,16.6,0.959461,0.993091,0.981481,16.431493,7.383116,32.596450,0.000135
4,01,Aguascalientes,01005,Jesús María,129929.0,0.494262,0.505738,0.075541,0.036274,0.039268,...,11.3,31.6,7.9,0.959461,0.993091,0.981481,12.356755,6.745819,34.731715,0.000256
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2452,32,Zacatecas,32054,Villa Hidalgo,19446.0,0.488738,0.511262,0.109328,0.054510,0.054818,...,16.7,76.8,38.5,0.991437,1.000000,0.824074,16.920279,9.895491,31.075955,0.000053
2453,32,Zacatecas,32055,Villanueva,31558.0,0.494011,0.505989,0.178655,0.086729,0.091926,...,14.7,68.2,28.7,0.991437,1.000000,0.824074,21.746589,12.050064,30.526273,0.000015
2454,32,Zacatecas,32056,Zacatecas,149607.0,0.481074,0.518926,0.125977,0.056261,0.069716,...,6.5,34.9,8.5,0.991437,1.000000,0.824074,20.010453,11.814172,36.822130,0.000339
2455,32,Zacatecas,32057,Trancoso,20455.0,0.490785,0.509215,0.087265,0.042728,0.044537,...,16.3,70.7,30.1,0.991437,1.000000,0.824074,15.941678,8.089966,32.477646,0.000097


Creamos un dataframe sólo con los nombres de las variables y un par de columnas vacías para poder hacer (a pie) una selección de variables

In [ ]:
datos = {'datos':indicadores_municipales.columns}
datos = pd.DataFrame(datos)
datos['Base Index'] = None
datos['DySoVi'] = None
datos

,datos,Base Index,DySoVi
0,entidad_cvegeo,None,None
1,State,None,None
2,municipio_cvegeo,None,None
3,Municipality,None,None
4,Population,None,None
5,Male,None,None
6,Female,None,None
7,60 and older,None,None
8,60 and older males,None,None
9,60 and older females,None,None


In [ ]:
datos.to_csv("datos/municipios/actualizacion_2020/prepara_diccionario_dysovi.csv")